In [2]:
def read_csv_file(file):
    while True:
        record = file.readline()
        if not record:
            break
        yield record
        

In [3]:
# get csv meta data in following dictionary structure
"""
{
"cosit_1":{
            records:[
                offset_line_1,
                offset_line_2,
                ..., 
                ..., 
                ..., 
                ..., 
                offset_line_n] 
        },
"cosit_2":{
            records:[
                offset_line_1,
                offset_line_2,
                ..., 
                ..., 
                ..., 
                ..., 
                offset_line_n] 
        },
...,
...,
...,

"cosit_n":{
            records:[
                offset_line_1,
                offset_line_2,
                ..., 
                ..., 
                ..., 
                ..., 
                offset_line_n] 
        }
}

"""
sites = {}
with open('./per-vehicle-records-2020-01-31.csv') as file:
    i=0
    offset=0
    for record in read_csv_file(file):
        
        cosit = record.split(',')[0]
#         ignore header
        if(i==0): 
            i = i+1
            offset = offset+len(record)
            continue

        else:
            a = cosit.strip("\"").lstrip("0")
            if(a in sites.keys()):
                sites[a]['records'].append(offset)
            else:
                sites[a] = {'records': [offset]}
            offset = offset+len(record)
            i = i+1
    

In [4]:
def yieldlines(thefile, offset):
    thefile.seek(offset)
    line = thefile.readline()
    yield line


In [5]:
import threading
import time
import string
import random


def sleeper(sites, startSiteIndex, endSiteIndex):
    with open('./per-vehicle-records-2020-01-31.csv') as f:
        key_list = list(sites.keys())
        while True:
            random_text = ''.join(random.choices(string.ascii_uppercase + string.digits, k=9))
            for index in range (startSiteIndex, endSiteIndex):
                cosit = key_list[index]
#                 ignoring test site
                if int(cosit) == 997:
                    continue
                counter_file =  open("./streams/countdata{}_{}.txt".format(cosit, random_text),'w')
                lines = ""
                if sites[cosit]['records'] == 0:
                    continue
                if len(sites[cosit]['records']) >= 10:
#                     if length of remaining records is greater than or equal to 10, get 10 records
#                     by looping 10 times
                    for i in range(0,10):
#                    poping the records, so that record offset is taken out from remaining records list 
#                     and also value is returned to yeild line from offset
                        line = next(yieldlines(f, sites[cosit]['records'].pop(0)))
                        lines = lines + line

                else:
#                     else get all remaining records
                    for i in range(0,len(sites[cosit]['records'])):
                        line = next(yieldlines(f, sites[cosit]['records'].pop(0)))
                        lines = lines + line

                counter_file.write(lines)
                counter_file.close()
#                 get data every 5 seconds
            time.sleep(5)
        
sleeper(sites, 0, 335)


KeyboardInterrupt: 